In [ ]:
"""
Custom notebook to load and plot multiple realisations of:
 - density PDF sigma VS time
 - RMS Mach number VS time
 - sigma VS Mach number (from interpolating the above two)
 
Plots the average between realisations of the same type, as well as an error bar/
shaded region corresponding the the 1-sigma standard deviation between simulations.

Requires 'sigma_vs_time.data' and 'rmsmach_vs_time.data' to already exist for 
each simulation. (Use 'gaussian_fit_pdf.py' to generate 'sigma_vs_time.data', and 
use 'rmsmach_vs_time.sh' to generate 'rmsmach_vs_time.data'.)
Directories for each can be specified in the first cell, as a list of strings.

Please note:
Since each simulation runs to different lengths, in order to take the average
and 1-sigma error and have no "kinks" in your plot, all simulations are chopped
off at the length of the shortest one.

Second note: we found an additional small kink in some of the compressive simulations
due to a small issue in some restarts. As such we plot only up to this point. This is
currently hard-coded.

Written by:
David Liptai, Monash University.
2015-2016
"""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp as kstest
import os
import subprocess
from scipy.interpolate import interp1d
%matplotlib

#Plotting stuff nicely
from matplotlib import gridspec
from IPython.display import display, Math, Latex
import math
from math import sqrt, cos, sin, pi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
from matplotlib.ticker import FormatStrFormatter, LinearLocator, NullFormatter, NullLocator, MultipleLocator
import matplotlib.ticker
import matplotlib.colors
from matplotlib.font_manager import FontProperties
from matplotlib import rc, text
plt.close('all')
fig_width_pt = 504   #245.27        # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0      # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]

fontsize=22
#fig_size = [18,16]
params = {'backend': 'pdf',
          'axes.labelsize': fontsize,
          'lines.markersize': 4,
          'font.size': fontsize,
          'xtick.major.size':8,
          'xtick.minor.size':4,
          'ytick.major.size':8,
          'ytick.minor.size':4,
          'xtick.major.width':2,
          'ytick.major.width':2,
          'xtick.minor.width':1,
          'ytick.minor.width':1,
          'lines.markeredgewidth':1,
          'axes.linewidth':1.2,
          'legend.fontsize': fontsize-3,
          'xtick.labelsize': fontsize-2,
          'ytick.labelsize': fontsize-2,
          'savefig.dpi':200,
          'path.simplify':True,
          'font.family': 'serif',
          'font.serif':'Times',
          'text.latex.preamble': [r'\usepackage{amsmath}',r'\DeclareMathAlphabet{\mathcal}{OMS}{cmsy}{m}{n}'],
          'text.usetex':True,
          'axes.color_cycle': ['b', 'lime', 'r', 'purple', 'g', 'c', 'm', 'orange', 'darkblue', \
                               'darkcyan', 'y','orangered','chartreuse','brown','deeppink','lightgreen', 'k'],
          #'font.serif':cm,
          'figure.figsize': fig_size}
plt.rcParams.update(params)
plt.clf()
gs = gridspec.GridSpec(1,1)
plt.close('all')

def BASH(command):
	return subprocess.check_output(command,shell=True).decode().strip()

utime       =  1.487E+13/60./60./24./365. 				# in years
t_ff        =  0.806588045/2.*utime						# in years


solenoidals = ['sol' +str(i) for i in range(1,8)]
compressives= ['comp'+str(i) for i in range(1,8)]
dir_names = solenoidals+compressives
dir_home = BASH('echo $HOME')
dir_prefix = '/Volumes/dlip1/runs/' #dir_home+'/dlip1/runs/'
#datafile='rmsmach_vs_time.data'
#datafile='sigma_vs_time.data'

In [ ]:
"""
Function to load a collection of data files and return a table of values and the length of the 
shortest simulation. (Simulations of shorter lenght as padded with zeros)
"""

def load_data_file(datafile,folders):
    # Input: datafile, list of folder names e.g. [sol1,sol2,....]
    # Output: data array
    L_shortest=1e10
    L_longest=0
    data=np.zeros([1e6,len(folders)+1])
    for i in range(0,len(folders)):
        name = folders[i]
        temp = np.genfromtxt(dir_prefix+name+'/'+datafile,skip_footer=1)
        L_shortest = min(len(temp),L_shortest)
        if len(temp)>L_longest:
            data[:len(temp),0] = temp[:len(temp),0] #Rewrite time column
        data[:len(temp),i+1] = temp[:len(temp),1]
        L_longest  = max(len(temp),L_longest)
    shortest_length=L_shortest
    data=data[:L_longest,:]
    return data, shortest_length

In [ ]:
sol_sig,lsol_sig = load_data_file(datafile='sigma_vs_time.data',folders=solenoidals)
sol_rms,lsol_rms = load_data_file(datafile='rmsmach_vs_time.data',folders=solenoidals)
comp_sig,lcomp_sig = load_data_file(datafile='sigma_vs_time.data',folders=compressives)
comp_rms,lcomp_rms = load_data_file(datafile='rmsmach_vs_time.data',folders=compressives)
    
'''    
L_shortest=1e10
L_longest=0
soldata=np.zeros([1e6,len(solenoidals)+1])
for i in range(0,len(solenoidals)):
    name = solenoidals[i]
    temp = np.genfromtxt(dir_prefix+name+'/'+datafile,skip_footer=1)
    L_shortest = min(len(temp),L_shortest)
    if len(temp)>L_longest:
        soldata[:len(temp),0] = temp[:len(temp),0] #Rewrite time column
    soldata[:len(temp),i+1] = temp[:len(temp),1]
    L_longest  = max(len(temp),L_longest)
lsol=L_shortest
soldata=soldata[:L_longest,:]

L_shortest=1e10
L_longest=0
compdata=np.zeros([1e6,len(compressives)+1])
for i in range(0,len(compressives)):
    name = compressives[i]
    temp = np.genfromtxt(dir_prefix+name+'/'+datafile,skip_footer=1)
    L_shortest = min(len(temp),L_shortest)
    if len(temp)>L_longest:
        compdata[:len(temp),0] = temp[:len(temp),0] #Rewrite time column
    compdata[:len(temp),i+1] = temp[:len(temp),1]
    L_longest  = max(len(temp),L_longest)
lcomp=L_shortest
compdata=compdata[:L_longest,:]
'''

print('')

In [ ]:
"""
Plot rms mach no. vs time and sigma vs time.
(Iterpolation is also done here)
"""

plt.close('all')
#f1 = plt.figure(figsize=fig_size)
plt.figure()
#gs = gridspec.GridSpec(1,1)
#ax0=f1.add_subplot(gs[0])

t1=sol_sig[:lsol_sig,0]
y1=sol_sig[:lsol_sig,1:]
y1_av=np.mean(y1,axis=1)
y1_std=np.std(y1,axis=1)

SOL_SIG=interp1d(t1,y1,axis=0)

t2=comp_sig[:lcomp_sig,0]
y2=comp_sig[:lcomp_sig,1:]
y2_av=np.mean(y2,axis=1)
y2_std=np.std(y2,axis=1)

nkink=np.argmax(t2>0.20418) #index beyond which kink occurs
t2=t2[:nkink]
y2=y2[:nkink]
y2_av=y2_av[:nkink]
y2_std=y2_std[:nkink]

COMP_SIG=interp1d(t2,y2,axis=0)

plt.plot(t1,y1_av,'g',label='Solenoidal')
plt.fill_between(t1, y1_av+y1_std, y1_av-y1_std,color='g',alpha=0.1)
plt.plot(t2,y2_av,'r',label='Compressive')
plt.fill_between(t2, y2_av+y2_std, y2_av-y2_std,color='r',alpha=0.1)
#plt.legend(loc='lower right',frameon=False)
plt.xlabel(r'Time [$t_{\mathrm{ff}}$]')
plt.ylabel(r'$\sigma$')
plt.xlim(xmax=max(max(t1),max(t2)))
plt.tick_params(which='both',axis='both',pad=8)
plt.tight_layout()
#plt.savefig('/Users/david/Desktop/sigma_time2.pdf')
plt.show()

plt.figure()
t1=sol_rms[:lsol_rms,0]
y1=sol_rms[:lsol_rms,1:]
y1_av=np.mean(y1,axis=1)
y1_std=np.std(y1,axis=1)

SOL_RMS=interp1d(t1,y1,axis=0)

t2=comp_rms[:lcomp_rms,0]
y2=comp_rms[:lcomp_rms,1:]
y2_av=np.mean(y2,axis=1)
y2_std=np.std(y2,axis=1)

np.argmax(t2>0.20418) #index beyond which kink occurs
t2=t2[:nkink]
y2=y2[:nkink]
y2_av=y2_av[:nkink]
y2_std=y2_std[:nkink]
COMP_RMS=interp1d(t2,y2,axis=0)

plt.plot(t1,y1_av,'g',label='Solenoidal')
plt.fill_between(t1, y1_av+y1_std, y1_av-y1_std,color='g',alpha=0.1)
plt.plot(t2,y2_av,'r',label='Compressive')
plt.fill_between(t2, y2_av+y2_std, y2_av-y2_std,color='r',alpha=0.1)
plt.legend(loc='upper right',frameon=False)
plt.xlabel(r'Time [$t_{\mathrm{ff}}$]')
plt.ylabel(r'$\mathcal{M}$')
plt.xlim(xmax=max(max(t1),max(t2)))
plt.tick_params(which='both',axis='both',pad=8)
plt.tight_layout()
#plt.savefig('/Users/david/Desktop/machnumber_time2.pdf')
plt.show()

print('')

In [ ]:
"""
Plot sigma vs mach number
(Previous cell needs to be run before this one for it to work; requires interpolated functions.)
"""

#plt.close('all')
plt.figure()
TIMEsol=min(np.max(sol_sig[:lsol_sig,0]),np.max(sol_rms[:lsol_rms,0]))
TIMEcomp=min(np.max(comp_sig[:lcomp_sig,0]),np.max(comp_rms[:lcomp_rms,0]))
timesol=np.linspace(0,TIMEsol,200)
timecomp=np.linspace(0,TIMEcomp,200)
if TIMEcomp>0.204:
    timecomp=np.linspace(0,0.204,200)

y=np.mean(SOL_SIG(timesol),axis=1)
y_std=np.std(SOL_SIG(timesol),axis=1)
x=np.mean(SOL_RMS(timesol),axis=1)
x_std=np.std(SOL_RMS(timesol),axis=1)
plt.plot(x,y,color='g',label='Solenoidal')
#plt.fill_between(x,y-y_std,y+y_std,color='g',alpha=0.1)
plt.fill_betweenx(y,x-x_std,x+x_std,color='g',alpha=0.1)

y=np.mean(COMP_SIG(timecomp),axis=1)
y_std=np.std(COMP_SIG(timecomp),axis=1)
x=np.mean(COMP_RMS(timecomp),axis=1)
x_std=np.std(COMP_RMS(timecomp),axis=1)
plt.plot(x,y,color='r',label='Compressive')
#plt.fill_between(x,y-y_std,y+y_std,color='r',alpha=0.1)
plt.fill_betweenx(y,x-x_std,x+x_std,color='r',alpha=0.1)

#plt.legend(loc='lower left',frameon=False)
plt.xlabel(r'$\mathcal{M}$')
plt.ylabel(r'$\sigma$')
plt.xlim(xmax=6.4)
plt.tick_params(which='both',axis='both',pad=8)
plt.tight_layout()
#plt.savefig('/Users/david/Desktop/sigma_machnumber.pdf')
plt.show()

In [ ]:
"""
#OLD STUFF....

#Double panel plot
from matplotlib import gridspec
from IPython.display import display, Math, Latex
import math
from math import sqrt, cos, sin, pi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
from matplotlib.ticker import FormatStrFormatter, LinearLocator, NullFormatter, NullLocator, MultipleLocator
import matplotlib.ticker
import matplotlib.colors
from matplotlib.font_manager import FontProperties
from matplotlib import rc, text
plt.close('all')
'''fig_width_pt = 2*513.17   #245.27        # Get this from LaTeX using \showthe\columnwidth
    inches_per_pt = 1.0/72.27               # Convert pt to inches
    golden_mean = (np.sqrt(5)-1.0)/2.0      # Aesthetic ratio
    fig_width = fig_width_pt*inches_per_pt  # width in inches
    fig_height =fig_width*golden_mean       # height in inches
    fig_size = [fig_width,fig_height]
    '''
fig_size = [15,11]
params = {'backend': 'pdf',
          'axes.labelsize': 25,
          'lines.markersize': 4,
          'font.size': 25,
          'xtick.major.size':8,
          'xtick.minor.size':4,
          'ytick.major.size':8,
          'ytick.minor.size':4,
          'xtick.major.width':2,
          'ytick.major.width':2,
          'xtick.minor.width':1,
          'ytick.minor.width':1,
          'lines.markeredgewidth':1,
          'axes.linewidth':1.2,
          'legend.fontsize': 25,
          'xtick.labelsize': 22,
          'ytick.labelsize': 22,
          'savefig.dpi':200,
          'path.simplify':True,
          'font.family': 'serif',
          'font.serif':'Times',
          'text.latex.preamble': [r'\usepackage{amsmath}'],
          'text.usetex':True,
          'axes.color_cycle': ['b', 'lime', 'r', 'purple', 'g', 'c', 'm', 'orange', 'darkblue', \
                               'darkcyan', 'y','orangered','chartreuse','brown','deeppink','lightgreen', 'k'],
          #'font.serif':cm,
          'figure.figsize': fig_size}
plt.rcParams.update(params)

plt.clf()
f1 = plt.figure(figsize=fig_size)
gs = gridspec.GridSpec(2,1)

ax0=f1.add_subplot(gs[0])
ax0.set_xlabel(r'$t_{\mathrm{ff}}$')
ax0.set_ylabel(r'$\mathcal{M}$')
#legend(loc='lower right',fontsize=fontsize_legend)
#show()
#ax0.legend(loc="upper right",fontsize=15,frameon=False,markerscale=0)
#ax0.ticklabel_format(style='sci',axis='y',scilimits=(0,0)) #force scienctific notation
#ax0.get_xaxis().set_ticks([])
ax0.tick_params(which='both',axis='both',pad=8)

ax1=f1.add_subplot(gs[1])
"""